In [18]:
import pandas as pd
import selfies as sf
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, r2_score
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem
from collections import Counter
import seaborn as sns
import torch
from torch.utils.data import Dataset
import torch.nn as nn
from tensorflow.keras.preprocessing.sequence import pad_sequences
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import csv
import selfies as sf
from collections import Counter
import torch.nn as nn
import csv
import os
import pandas as pd
from cddd.inference import InferenceModel
from cddd.preprocessing import preprocess_smiles
from scipy.ndimage import gaussian_filter

In [19]:
!python3 example/run_qsar_test.py --model_dir cddd/pretrained_models/default_model


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

W0627 08:23:51.690348 130154582922752 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Traceback (most recent call last):
  File "example/run_qsar_test.py", line 94, in <module>
    tf.app.run(main=main, argv=[sys.argv[0]] + UNPARSED)
  File "/home/undergrad/2026/melkoudi/miniconda3/envs/tf110/lib/p

In [20]:
!cddd --input smiles.smi --output descriptors.csv  --smiles_header smiles


/home/undergrad/2026/melkoudi/miniconda3/envs/tf110/lib/python3.6/site-packages/cddd/run_cddd.py:55: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  sml_df = pd.read_table(file, header=None).rename({0:FLAGS.smiles_header, 1:"EXTREG"},
Traceback (most recent call last):
  File "/home/undergrad/2026/melkoudi/miniconda3/envs/tf110/bin/cddd", line 8, in <module>
    sys.exit(main_wrapper())
  File "/home/undergrad/2026/melkoudi/miniconda3/envs/tf110/lib/python3.6/site-packages/cddd/run_cddd.py", line 99, in main_wrapper
    tf.app.run(main=main, argv=[sys.argv[0]] + UNPARSED)
  File "/home/undergrad/2026/melkoudi/miniconda3/envs/tf110/lib/python3.6/site-packages/tensorflow_core/python/platform/app.py", line 40, in run
    _run(main=main, argv=argv, flags_parser=_parse_flags_tolerate_undef)
  File "/home/undergrad/2026/melkoudi/miniconda3/envs/tf110/lib/python3.6/site-packages/absl/app.py", line 308, in run
    _run_main(main, args)
  File "/home/undergra

# Smiles to Embedding via cddd

In [21]:
# create a smiles list of 6000
smiles_list = []

with open("computed_spectra.csv", "r") as f:
    reader = csv.reader(f)
    next(reader)  # skip header line
    for i, row in enumerate(reader):
        if i >= 6000:
            break
        smiles = row[0].strip()
        try:
            smiles_list.append(smiles)
        except:
            continue

# make instance of autoencoder model
inference_model = InferenceModel(model_dir="cddd/data/default_model")

# embed the smiles
# get 512-dim CDDD embeddings
smiles_embedding = inference_model.seq_to_emb(smiles_list)


# print example to see if it works
for i in range(5):
    print(f"SMILES: {smiles_list[i]}") # full smiles
    print(f"Embedding: {smiles_embedding[i][:10]}...\n")  # print first 10 values of the embedding



INFO:tensorflow:Restoring parameters from cddd/data/default_model/model.ckpt
SMILES: CN(c1ccc(NC(=O)Nc2ccccc2)cc1)S(=O)(=O)c1ccc(-c2ccn(CCO)n2)s1
Embedding: [-0.6485799  -0.4820745  -0.29993665  0.2840209  -0.19669785  0.23079348
 -0.45663777 -0.50177956 -0.06138841  0.28148285]...

SMILES: CC(O)CC(C)C#COC#CC(C)CC(C)O
Embedding: [-0.23311913 -0.62234724 -0.14250664  0.38820195  0.27778637 -0.15076475
  0.01024481 -0.58167255 -0.23104398 -0.5806556 ]...

SMILES: Cc1cc([N+](=O)[O-])ccc1NC(=O)c1ccc(OCC(C)C)c(Br)c1
Embedding: [-0.47915938 -0.5277588   0.254828   -0.35512787 -0.01809985  0.02691496
 -0.17222045 -0.12126444  0.45158276  0.21312533]...

SMILES: COC12C(COC(N)=O)C3=C(C(=O)C(C)=C(N)C3=O)N1CC1NC12
Embedding: [ 0.95607096 -0.4094612   0.5092193   0.33736256  0.11703882  0.35932195
  0.7206423  -0.6761475  -0.16922899  0.69493043]...

SMILES: CC(C)(O)C(NC(=O)c1cnn2cc(C3CC3)cnc12)c1ccc(OC(F)(F)F)c(F)c1
Embedding: [-0.8880903   0.08556144  0.20473011  0.1794778   0.02583974 -0.440634

# Normalize Spectra 

In [22]:
import numpy as np
import csv
from sklearn.preprocessing import MinMaxScaler

spec_len = 1801
spectra_list = []

# Load spectra from CSV
with open("computed_spectra.csv", "r") as f:
    reader = csv.reader(f)
    next(reader)  # skip header
    for i, row in enumerate(reader):
        if i >= 6000:
            break
        try:
            spectrum = [float(x) for x in row[1:]]
            if len(spectrum) == spec_len:
                spectra_list.append(spectrum)
        except:
            continue

# convert to np array
spectra_array = np.array(spectra_list)

# normalize using minmax scaling
scaler = MinMaxScaler()
normalized_spectra = scaler.fit_transform(spectra_array)

# the input for the model
X = normalized_spectra

# check if it worked :)))
print("Normalized spectra shape:", X.shape)
print("Sample normalized spectrum:", X[0][:10])


Normalized spectra shape: (3568, 1801)
Sample normalized spectrum: [0.10808033 0.10908509 0.1097319  0.11094673 0.11241841 0.11278684
 0.11332587 0.11278429 0.11410056 0.11418134]


# Train/Test Split 

In [23]:
from sklearn.model_selection import train_test_split

# set up X and Y
X = np.array(normalized_spectra)
Y = np.array(smiles_embedding)

# fit to right length
min_len = min(len(X), len(Y))
X = X[:min_len]
Y = Y[:min_len]

# debug rqqq
print(f" shape: {X.shape}, Y shape: {Y.shape}")


# train/test split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

print("Train size:", len(X_train), "Test size:", len(X_test))


 shape: (3568, 1801), Y shape: (3568, 512)
Train size: 2854 Test size: 714


# Dataset and Dataloaders

In [24]:
import torch
from torch.utils.data import Dataset, DataLoader

class SpectraToSMILESDataset(Dataset):
    def __init__(self, X, Y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.Y = torch.tensor(Y, dtype=torch.float32)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.Y[idx]

# wrap train/test sets
train_dataset = SpectraToSMILESDataset(X_train, Y_train)
test_dataset = SpectraToSMILESDataset(X_test, Y_test)

# loaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64)


# Model:

In [25]:
import torch.nn as nn

class SpectraToEmbedding(nn.Module):
    def __init__(self, input_dim=1801, output_dim=512, hidden_dim=1024):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.BatchNorm1d(hidden_dim),
            
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.3),
            
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),

            nn.Linear(hidden_dim, output_dim)
        )


    def forward(self, x):
        return self.model(x)

model = SpectraToEmbedding()

# Model (2nd Try)

In [28]:
import torch.nn as nn

class SpectraToEmbedding(nn.Module):
    def __init__(self, input_dim=1801, output_dim=512, hidden_dim=1024):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.BatchNorm1d(hidden_dim),

            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.3),

            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.BatchNorm1d(hidden_dim),

            nn.Linear(hidden_dim, output_dim)
        )

    def forward(self, x):
        return self.model(x)

model = SpectraToEmbedding()


# Train 2nd Model

In [29]:
import torch
import torch.nn.functional as F

# Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# Scheduler (optional but helpful for long training)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.5, patience=5, verbose=True)

# Hybrid loss function (MSE + Cosine)
def hybrid_loss(y_pred, y_true):
    mse = F.mse_loss(y_pred, y_true)
    cos = 1 - F.cosine_similarity(y_pred, y_true, dim=1).mean()
    return 0.5 * mse + 0.5 * cos

# Training loop
num_epochs = 200
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for X_batch, Y_batch in train_loader:
        optimizer.zero_grad()
        Y_pred = model(X_batch)

        loss = hybrid_loss(Y_pred, Y_batch)

        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    scheduler.step(avg_loss)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}")


Epoch 1/200, Loss: 0.4123
Epoch 2/200, Loss: 0.2724
Epoch 3/200, Loss: 0.2575
Epoch 4/200, Loss: 0.2478
Epoch 5/200, Loss: 0.2417
Epoch 6/200, Loss: 0.2362
Epoch 7/200, Loss: 0.2306
Epoch 8/200, Loss: 0.2254
Epoch 9/200, Loss: 0.2206
Epoch 10/200, Loss: 0.2156
Epoch 11/200, Loss: 0.2116
Epoch 12/200, Loss: 0.2063
Epoch 13/200, Loss: 0.2028
Epoch 14/200, Loss: 0.1973
Epoch 15/200, Loss: 0.1948
Epoch 16/200, Loss: 0.1895
Epoch 17/200, Loss: 0.1862
Epoch 18/200, Loss: 0.1858
Epoch 19/200, Loss: 0.1822
Epoch 20/200, Loss: 0.1740
Epoch 21/200, Loss: 0.1711
Epoch 22/200, Loss: 0.1681
Epoch 23/200, Loss: 0.1652
Epoch 24/200, Loss: 0.1631
Epoch 25/200, Loss: 0.1575
Epoch 26/200, Loss: 0.1543
Epoch 27/200, Loss: 0.1527
Epoch 28/200, Loss: 0.1495
Epoch 29/200, Loss: 0.1473
Epoch 30/200, Loss: 0.1458
Epoch 31/200, Loss: 0.1428
Epoch 32/200, Loss: 0.1407
Epoch 33/200, Loss: 0.1375
Epoch 34/200, Loss: 0.1359
Epoch 35/200, Loss: 0.1350
Epoch 36/200, Loss: 0.1337
Epoch 37/200, Loss: 0.1305
Epoch 38/2

# Normalize embeddings

In [30]:
predicted_embeddings = predicted_embeddings / np.linalg.norm(predicted_embeddings, axis=1, keepdims=True)

# Train za Model

In [26]:
# optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# loss function
loss_fn = nn.CosineEmbeddingLoss()

# Training loop
num_epochs = 200
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for X_batch, Y_batch in train_loader:
        optimizer.zero_grad()
        Y_pred = model(X_batch)

        # ADD THIS: target = tensor of 1s indicating "similar pairs"
        target = torch.ones(X_batch.size(0)).to(X_batch.device)

        # Now compute loss correctly
        loss = loss_fn(Y_pred, Y_batch, target)

        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}")


Epoch 1/200, Loss: 0.5194
Epoch 2/200, Loss: 0.4579
Epoch 3/200, Loss: 0.4199
Epoch 4/200, Loss: 0.3985
Epoch 5/200, Loss: 0.3776
Epoch 6/200, Loss: 0.3640
Epoch 7/200, Loss: 0.3553
Epoch 8/200, Loss: 0.3459
Epoch 9/200, Loss: 0.3387
Epoch 10/200, Loss: 0.3308
Epoch 11/200, Loss: 0.3256
Epoch 12/200, Loss: 0.3181
Epoch 13/200, Loss: 0.3141
Epoch 14/200, Loss: 0.3076
Epoch 15/200, Loss: 0.3043
Epoch 16/200, Loss: 0.2988
Epoch 17/200, Loss: 0.2940
Epoch 18/200, Loss: 0.2880
Epoch 19/200, Loss: 0.2834
Epoch 20/200, Loss: 0.2774
Epoch 21/200, Loss: 0.2744
Epoch 22/200, Loss: 0.2685
Epoch 23/200, Loss: 0.2654
Epoch 24/200, Loss: 0.2610
Epoch 25/200, Loss: 0.2558
Epoch 26/200, Loss: 0.2503
Epoch 27/200, Loss: 0.2469
Epoch 28/200, Loss: 0.2420
Epoch 29/200, Loss: 0.2375
Epoch 30/200, Loss: 0.2344
Epoch 31/200, Loss: 0.2302
Epoch 32/200, Loss: 0.2260
Epoch 33/200, Loss: 0.2226
Epoch 34/200, Loss: 0.2196
Epoch 35/200, Loss: 0.2168
Epoch 36/200, Loss: 0.2125
Epoch 37/200, Loss: 0.2102
Epoch 38/2

# Eval

In [31]:
# remove any warnings
from rdkit import RDLogger
RDLogger.DisableLog('rdApp.*')

# eval
model.eval()
predicted_embeddings = []

# pred embeddings from test spectra
with torch.no_grad():
    for X_batch, _ in test_loader:
        preds = model(X_batch)
        predicted_embeddings.append(preds)

predicted_embeddings = torch.cat(predicted_embeddings, dim=0).cpu().numpy()

# decode embeddings
cddd = InferenceModel(model_dir="cddd/data/default_model")
pred_smiles = cddd.emb_to_seq(predicted_embeddings)
true_smiles = cddd.emb_to_seq(Y_test)

# eval metrics
tanimoto_scores = []
valid_count = 0
exact_match_count = 0
total = len(true_smiles)

for pred, true in zip(pred_smiles, true_smiles):
    try:
        pred_mol = Chem.MolFromSmiles(pred)
        true_mol = Chem.MolFromSmiles(true)
        
        # validity check
        if pred_mol is not None:
            valid_count += 1

        # exact match?
        if pred == true:
            exact_match_count += 1

        # tanimoto sim
        if pred_mol is not None and true_mol is not None:
            fp_pred = AllChem.GetMorganFingerprintAsBitVect(pred_mol, 2, nBits=2048)
            fp_true = AllChem.GetMorganFingerprintAsBitVect(true_mol, 2, nBits=2048)
            similarity = DataStructs.TanimotoSimilarity(fp_pred, fp_true)
            tanimoto_scores.append(similarity)
        else:
            tanimoto_scores.append(0.0)
    except:
        tanimoto_scores.append(0.0)

# print final metrics
valid_percent = 100 * valid_count / total
exact_match_percent = 100 * exact_match_count / total
avg_tanimoto = np.mean(tanimoto_scores)

print(f"Average Tanimoto Similarity: {avg_tanimoto:.4f}")
print(f"Valid SMILES: {valid_count}/{total} ({valid_percent:.2f}%)")
print(f"Exact Matches: {exact_match_count}/{total} ({exact_match_percent:.2f}%)")

INFO:tensorflow:Restoring parameters from cddd/data/default_model/model.ckpt
INFO:tensorflow:Restoring parameters from cddd/data/default_model/model.ckpt
Average Tanimoto Similarity: 0.2070
Valid SMILES: 708/714 (99.16%)
Exact Matches: 0/714 (0.00%)
